In [1]:
# Setting up fastai and kaggle reqs

!pip install -q fastkaggle
!pip install -q timm
!pip install --ignore-installed PyYAML
!pip install -q fastai

!pip install -q lightning
from fastkaggle import setup_comp
from pathlib import Path
import timm
from fastai.vision.all import *
import lightning.pytorch as pl

# set_seed(42)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.8/661.8 kB 4.9 MB/s eta 0:00:00 MB/s eta 0:00:01:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.20 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.20 requires nbformat==5.4.0, but you have nbformat 5.5.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
triton 2.0.0 requires cmake, which is not installed.
triton 2.0.0 requires lit, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requi

/home/ktk/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/ktk/anaconda3/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator9allocatorE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


ContextualVersionConflict: (Pygments 2.11.2 (/home/ktk/anaconda3/lib/python3.9/site-packages), Requirement.parse('pygments<3.0.0,>=2.13.0'), {'rich'})

In [ ]:
creds = '{"username":"kayteekay22","key":"c95053f45a6668540c1582e2bf18e29b"}'
comp = 'dog-breed-identification'

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

path = setup_comp(comp, install='fastai "timm>=0.6.2.dev0"')


In [ ]:
# Create label finder
import pandas as pd
label_path = path/'labels.csv'
label_df = pd.read_csv(label_path)
def get_lab(id_str): 
    # print(type(label_df.loc[label_df['id'] == str(id_str.stem)]['breed'].values[0]))
    return str(label_df.loc[label_df['id'] == str(id_str.stem)]['breed'].values[0])

In [ ]:
training_path_sml = Path('small')
resize_images(path/'train', dest=training_path_sml, max_size=256,  recurse=True)
# files = get_image_files(training_path_sml)
# # dls = ImageDataLoaders.from_folder(training_path_sml, valid_pct=0.2, seed=42, item_tfms=Resize(256,192))
# dls = ImageDataLoaders.from_path_func(training_path,files, label_func=get_lab, valid_pct=0.2, seed=42, item_tfms=Resize(256,192))
# dls.show_batch(max_n=3)

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader


In [ ]:
from torchvision.io import read_image
import os
from torch.utils.data import Dataset

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
class DogCNN(pl.LightningModule):
    def __init__(self, pre_trained_model, train_path, label_path, test_path=None) -> None:
        super(DogCNN, self).__init__()
        self.model = pre_trained_model
        self.train_path = train_path
        self.label_path = label_path
        self.test_path  = test_path
    
    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = nn.CrossEntropyLoss(outputs, labels)
        tensorboard_logs = {'train_loss': loss}
        # use key 'log'
        return {"loss": loss, 'log': tensorboard_logs}
    
    def train_dataloader(self):
        train_dataset = CustomImageDataset(self.train_path, transform = transforms.Compose([
        # resize
        transforms.Resize(256),
        # center-crop
        transforms.CenterCrop(256),
        # to-tensor
        transforms.ToTensor(),
        # normalize
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ]))
        
        train_dl = DataLoader(
        train_dataset,
        batch_size=8,
        shuffle=False,
        num_workers=0,
        collate_fn=None,
        pin_memory=False,
        )
        
        return train_dl
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

In [ ]:
from torchvision.models import ConvNeXt, ConvNeXt_Base_Weights
from torchvision import models 
pre_model = models.get_model("convnext_base", weights="DEFAULT")

label_path = path/'labels.csv'
model = DogCNN(pre_model, training_path_sml, label_path)

# model = LitNeuralNet(input_size, hidden_size, num_classes)

In [ ]:
trainer = Trainer(fast_dev_run=True)
trainer.fit(model)

In [6]:
# training_path = path/'train'
# files = get_image_files(training_path)
# # print(files)

In [6]:
def train(arch, item, batch, epochs=5, lr=0.001):
    dls = ImageDataLoaders.from_path_func(training_path_sml, files, label_func=get_lab, valid_pct=0.2, seed=42, item_tfms=item,
                                       batch_tfms=batch)
    learn  = vision_learner(dls, arch, metrics=error_rate).to_fp16()
    learn.lr_find()
    learn.fine_tune(epochs, lr)
    return learn

In [16]:
learn = train('convnext_large_in22k', item=Resize(256),
              batch=aug_transforms( size=128, min_scale=0.75),epochs=5)

TypeError: unsupported operand type(s) for *: 'float' and 'RandomErasing'

In [18]:
# /learn.fine_tune(2, 0.001)
aug_transforms??

Signature:
aug_transforms(
    mult: 'float' = 1.0,
    do_flip: 'bool' = True,
    flip_vert: 'bool' = False,
    max_rotate: 'float' = 10.0,
    min_zoom: 'float' = 1.0,
    max_zoom: 'float' = 1.1,
    max_lighting: 'float' = 0.2,
    max_warp: 'float' = 0.2,
    p_affine: 'float' = 0.75,
    p_lighting: 'float' = 0.75,
    xtra_tfms: 'list' = None,
    size: 'int | tuple' = None,
    mode: 'str' = 'bilinear',
    pad_mode='reflection',
    align_corners=True,
    batch=False,
    min_scale=1.0,
)
Source:   
def aug_transforms(
    mult:float=1.0, # Multiplication applying to `max_rotate`,`max_lighting`,`max_warp`
    do_flip:bool=True, # Random flipping
    flip_vert:bool=False, # Flip vertically
    max_rotate:float=10., # Maximum degree of rotation
    min_zoom:float=1., # Minimum zoom 
    max_zoom:float=1.1, # Maximum zoom 
    max_lighting:float=0.2, # Maximum scale of changing brightness 
    max_warp:float=0.2, # Maximum value of changing warp per
    p_affine:float=0.75, # 

In [9]:
# Getting all sizes for images to check if we can transform the batch

from fastcore.parallel import *
def f(o): return PILImage.create(o).size
sizes = parallel(f, files, n_workers=8)
pd.Series(sizes).value_counts()


(256, 192)    3050
(256, 170)    1121
(192, 256)     694
(170, 256)     409
(256, 171)     236
              ... 
(176, 195)       1
(120, 120)       1
(180, 209)       1
(175, 250)       1
(180, 156)       1
Length: 740, dtype: int64

In [8]:
test_files = get_image_files(path/'test').sorted()
test_dl = learn.dls.test_dl(test_files)

In [9]:
probs,_,idxs = learn.get_preds(dl=test_dl, with_decoded=True)
probs

TensorBase([[1.9241e-07, 1.7355e-08, 1.0331e-06,  ..., 3.7319e-08,
             2.4430e-07, 1.3854e-06],
            [1.6635e-06, 2.3356e-06, 1.2546e-05,  ..., 6.1919e-07,
             2.0539e-07, 4.0096e-06],
            [3.0325e-07, 1.8890e-05, 4.0337e-06,  ..., 1.5454e-06,
             8.1087e-06, 1.3899e-06],
            ...,
            [3.4228e-07, 5.8096e-07, 3.7631e-05,  ..., 4.3125e-08,
             2.1545e-07, 6.1413e-08],
            [3.7231e-05, 1.4587e-05, 1.9979e-05,  ..., 2.0826e-05,
             4.4625e-05, 1.4639e-04],
            [2.1107e-07, 1.3329e-05, 1.7727e-05,  ..., 8.8976e-05,
             4.6645e-07, 3.9781e-07]])

In [11]:
probs.size()
len(learn.dls.vocab)
# print(test_dl.items)
sub_df = pd.DataFrame(probs, columns=learn.dls.vocab)
ids  = [test_dl_val.stem for test_dl_val in test_dl.items]
sub_df.insert(0,'id', ids)

In [12]:
sub_df

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,1.924119e-07,1.735494e-08,1.033083e-06,1.373093e-07,8.574552e-09,3.252327e-07,4.523408e-07,3.129260e-07,5.569512e-08,...,4.511276e-07,6.240774e-07,2.229839e-07,7.510371e-07,2.957945e-08,1.044438e-08,7.949981e-07,3.731897e-08,2.443027e-07,1.385421e-06
1,00102ee9d8eb90812350685311fe5890,1.663515e-06,2.335645e-06,1.254648e-05,2.197680e-05,5.593014e-06,6.143681e-07,1.131269e-05,9.410658e-06,1.582930e-05,...,3.430530e-05,3.127645e-06,2.851928e-06,6.966142e-06,1.417544e-05,2.092949e-05,9.155085e-04,6.191867e-07,2.053860e-07,4.009608e-06
2,0012a730dfa437f5f3613fb75efcd4ce,3.032505e-07,1.889014e-05,4.033742e-06,8.525660e-07,1.323950e-06,3.608230e-07,1.676345e-07,6.022029e-07,5.395476e-07,...,6.400078e-08,1.143168e-07,3.909053e-07,9.356750e-07,2.046504e-05,8.923069e-06,9.918956e-07,1.545400e-06,8.108694e-06,1.389946e-06
3,001510bc8570bbeee98c8d80c8a95ec1,4.082353e-02,7.774943e-04,9.179033e-04,3.902441e-03,2.111386e-03,4.371595e-03,5.772208e-04,4.610565e-03,1.313748e-02,...,2.060770e-02,8.597678e-04,3.637475e-03,6.618857e-03,1.204207e-03,2.609920e-03,1.344702e-03,1.661297e-03,8.464383e-04,8.432643e-03
4,001a5f3114548acdefa3d4da05474c2e,1.094125e-04,1.025830e-04,4.821355e-06,6.014940e-06,7.290916e-06,4.908052e-06,8.602107e-07,1.329522e-07,3.817162e-07,...,8.936046e-07,1.536719e-05,1.316995e-05,2.266353e-06,1.756440e-06,7.487134e-08,1.426582e-06,6.050287e-06,5.672629e-06,3.766454e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10352,ffeda8623d4eee33c6d1156a2ecbfcf8,3.687365e-07,1.014489e-07,2.585533e-07,2.242302e-06,6.413941e-08,8.826952e-08,4.114948e-08,8.200774e-07,3.861863e-08,...,1.502697e-05,2.453917e-07,6.712914e-07,4.858798e-07,7.935643e-08,2.277829e-07,1.358826e-07,2.852855e-07,2.129919e-07,8.068487e-08
10353,fff1ec9e6e413275984966f745a313b0,1.015773e-05,1.364408e-04,7.833967e-06,8.348608e-05,2.880275e-05,1.128758e-05,3.356211e-06,1.557590e-05,1.632982e-04,...,5.878841e-06,1.291410e-06,1.026896e-04,1.716116e-05,9.737771e-01,6.688671e-05,1.815680e-05,1.832429e-04,1.407165e-05,2.067702e-06
10354,fff74b59b758bbbf13a5793182a9bbe4,3.422813e-07,5.809622e-07,3.763103e-05,2.456948e-07,2.580908e-06,4.250410e-07,1.190134e-07,1.374732e-06,2.459649e-07,...,2.645106e-07,1.027617e-06,7.862469e-08,9.567670e-08,1.466752e-07,3.636457e-07,1.129551e-07,4.312487e-08,2.154529e-07,6.141263e-08
10355,fff7d50d848e8014ac1e9172dc6762a3,3.723071e-05,1.458688e-05,1.997931e-05,6.727451e-06,4.841594e-05,1.333348e-05,5.265775e-05,7.331161e-06,1.616746e-06,...,2.060402e-04,5.252935e-05,1.879685e-04,2.396763e-05,1.045025e-05,4.463311e-07,2.206924e-05,2.082598e-05,4.462464e-05,1.463900e-04


In [13]:
sub_df.to_csv('subm.csv', index=False)
!head subm.csv

id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,bedlington_terrier,bernese_mountain_dog,black-and-tan_coonhound,blenheim_spaniel,bloodhound,bluetick,border_collie,border_terrier,borzoi,boston_bull,bouvier_des_flandres,boxer,brabancon_griffon,briard,brittany_spaniel,bull_mastiff,cairn,cardigan,chesapeake_bay_retriever,chihuahua,chow,clumber,cocker_spaniel,collie,curly-coated_retriever,dandie_dinmont,dhole,dingo,doberman,english_foxhound,english_setter,english_springer,entlebucher,eskimo_dog,flat-coated_retriever,french_bulldog,german_shepherd,german_short-haired_pointer,giant_schnauzer,golden_retriever,gordon_setter,great_dane,great_pyrenees,greater_swiss_mountain_dog,groenendael,ibizan_hound,irish_setter,irish_terrier,irish_water_spaniel,irish_wolfhound,italian_greyhound,japanese_spaniel,keeshond,kelpie,kerry_blue_terrier,komondor,kuvasz,labrador_retriever,lakeland_terrier,leonberg,lhasa,mala

In [14]:
from kaggle import api
api.competition_submit_cli('subm.csv', 'convnext submission', comp)

100%|██████████| 16.4M/16.4M [00:00<00:00, 21.6MB/s]


Successfully submitted to Dog Breed Identification